# AnchorViewer

In [1]:
# imports
import logging
import pathlib
import sys
import warnings
from typing import List, Dict

import numpy as np
from torchvision.models.detection.transform import GeneralizedRCNNTransform

from pytorch_faster_rcnn_tutorial.datasets import ObjectDetectionDataSet
from pytorch_faster_rcnn_tutorial.transformations import (
    Clip,
    ComposeDouble,
    FunctionWrapperDouble,
    normalize_01,
)
from pytorch_faster_rcnn_tutorial.utils import get_filenames_of_path
from pytorch_faster_rcnn_tutorial.viewers.anchor_viewer import AnchorViewer

In [2]:
warnings.filterwarnings("ignore")

logger: logging.Logger = logging.getLogger(__name__)

# logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(filename)s:%(lineno)d:%(funcName)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

In [3]:
# root directory
data_path: pathlib.Path = pathlib.Path("src/pytorch_faster_rcnn_tutorial/data/heads")

In [4]:
# input and target files
inputs: List[pathlib.Path] = get_filenames_of_path(data_path / "input")
targets: List[pathlib.Path] = get_filenames_of_path(data_path / "target")

inputs.sort()
targets.sort()

2023-06-08 17:42:58 - INFO - utils.py:27:get_filenames_of_path: Found 20 files in src/pytorch_faster_rcnn_tutorial/data/heads/input
2023-06-08 17:42:58 - INFO - utils.py:27:get_filenames_of_path: Found 20 files in src/pytorch_faster_rcnn_tutorial/data/heads/target


In [5]:
# mapping
mapping: Dict[str, int] = {
    "head": 1,
}

In [6]:
# transforms
transforms: ComposeDouble = ComposeDouble(
    [
        Clip(),
        # AlbumentationWrapper(albumentation=A.HorizontalFlip(p=0.5)),
        # AlbumentationWrapper(albumentation=A.RandomScale(p=0.5, scale_limit=0.5)),
        # AlbumentationWrapper(albumentation=A.VerticalFlip(p=0.5)),
        FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
        FunctionWrapperDouble(normalize_01),
    ]
)

In [7]:
# dataset building
dataset: ObjectDetectionDataSet = ObjectDetectionDataSet(
    inputs=inputs,
    targets=targets,
    transform=transforms,
    use_cache=False,
    convert_to_format=None,
    mapping=mapping,
)

In [8]:
# transforms
transform: GeneralizedRCNNTransform = GeneralizedRCNNTransform(
    min_size=1024,
    max_size=1024,
    image_mean=[0.485, 0.456, 0.406],
    image_std=[0.229, 0.224, 0.225],
)

In [9]:
feature_map_size = (512, 32, 32)

anchor_viewer_rcnn: AnchorViewer = AnchorViewer(
    dataset=dataset, rcnn_transform=transform, feature_map_size=feature_map_size
)

In [10]:
## press 'n' or 'b' to see images